In [2]:
import pandas as pd
import numpy as np

c:\Users\juziu\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\juziu\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Users\juziu\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
# Name, Supply, Cost
suppliers = [('D1', 20, 10), ('D2', 30, 12)]

# Name, Demand, Price
receivers = [('O1', 10, 30), ('O2', 28, 25), ('O3', 27, 30)]

# rows:    suppliers
# columns: receivers
transport_cost_matrix = np.array([
    [8, 14, 17],
    [12, 9, 19]
])

In [4]:
def unit_profits_matrix(transport_cost_matrix, suppliers, receivers):
    un_pr_matrix = np.empty_like(transport_cost_matrix)
    # Suppliers
    for i, row in enumerate(transport_cost_matrix):
        # Receivers
        for j, num in enumerate(row):
            
            un_pr_matrix[i, j] = receivers[j][2] - suppliers[i][2] - transport_cost_matrix[i, j]
    
    return un_pr_matrix

def supply_demand_sum(suppliers, receivers, supp_idx=1, rec_idx=1):
    supp_sum = sum([supp[supp_idx] for supp in suppliers])
    dmnd_sum = sum([recv[rec_idx] for recv in receivers])
    return supp_sum, dmnd_sum

In [5]:
unit_profits = unit_profits_matrix(transport_cost_matrix, suppliers, receivers)

In [6]:
indices = np.argsort(unit_profits, axis=None)[::-1]
indices = np.unravel_index(indices, unit_profits.shape)
indices

(array([0, 1, 1, 0, 0, 1], dtype=int64),
 array([0, 0, 1, 2, 1, 2], dtype=int64))

In [7]:
supp_sum, dmnd_sum = supply_demand_sum(suppliers, receivers)

if supp_sum != dmnd_sum:
    unit_profits = np.pad(unit_profits, ((0, 1), (0, 1)), 'constant', constant_values=0)
    transport_cost_matrix = np.pad(transport_cost_matrix, ((0, 1), (0, 1)), 'constant', constant_values=0)
    suppliers.append(('DF', dmnd_sum, 0))
    receivers.append(('OF', supp_sum, 0))

In [8]:
unit_profits, suppliers, receivers

(array([[12,  1,  3,  0],
        [ 6,  4, -1,  0],
        [ 0,  0,  0,  0]]),
 [('D1', 20, 10), ('D2', 30, 12), ('DF', 65, 0)],
 [('O1', 10, 30), ('O2', 28, 25), ('O3', 27, 30), ('OF', 50, 0)])

In [9]:
def sales_matrix(unit_profits, suppliers, receivers, supp_idx=1, rec_idx=1):
    suppliers_cpy = [list(tup) for tup in suppliers]
    receivers_cpy = [list(tup) for tup in receivers]
    # We start from REAL Suppliers and Receivers:
    sales = np.zeros(unit_profits.shape)
    for idx, jdx in zip(*indices):
        # Each iteration is one supplier-receiver transaction
        unit_profit = unit_profits[idx, jdx]
        supp_name, supply, cost = suppliers_cpy[idx]
        recv_name, demand, price = receivers_cpy[jdx]
        if supply == 0 or demand == 0:
            continue
        
        supply_after = supply - demand
        if supply_after >= 0:
            # no supply
            sales[idx, jdx] = demand
            
            # Update
            suppliers_cpy[idx][supp_idx] = supply_after
            receivers_cpy[jdx][rec_idx]  = 0
        if supply_after < 0:
            # no demand
            sales[idx, jdx] = supply
            
            # Update
            suppliers_cpy[idx][supp_idx] = 0
            receivers_cpy[jdx][rec_idx]  = -supply_after
    
    # We go to FICTIONAL Suppliers and Receivers
    for idx, row in enumerate(sales):
        for jdx, num in enumerate(row):
            # Each iteration is one supplier-receiver transaction
            unit_profit = unit_profits[idx, jdx]
            supp_name, supply, cost = suppliers_cpy[idx]
            recv_name, demand, price = receivers_cpy[jdx]
            if supply == 0 or demand == 0:
                continue
            
            supply_after = supply - demand
            if supply_after >= 0:
                # no supply
                sales[idx, jdx] = demand
                
                # Update
                suppliers_cpy[idx][supp_idx] = supply_after
                receivers_cpy[jdx][rec_idx]  = 0
            if supply_after < 0:
                # no demand
                sales[idx, jdx] = supply
                
                # Update
                suppliers_cpy[idx][supp_idx] = 0
                receivers_cpy[jdx][rec_idx]  = -supply_after
            
    if (sum([supp[supp_idx] for supp in suppliers_cpy]) or
        sum([recv[rec_idx] for recv in receivers_cpy])):
        raise Exception("Supply and Demand after calculations are not zero.")
    return sales, suppliers_cpy, receivers_cpy

In [10]:
sales, supp, recv = sales_matrix(unit_profits, suppliers, receivers)

In [11]:
mask = sales[:-1, :-1] !=0
masked_array = (unit_profits[:-1, :-1] * mask)

A = []
B = []
for i in range(mask.shape[0]):
    for j in range(mask.shape[1]):
        if mask[i, j]:
            row = [0]*(mask.shape[0] + mask.shape[1])
            row[i] = 1
            row[mask.shape[0] + j] = 1
            A.append(row)
            B.append(masked_array[i, j])

A = np.array(A)
B = np.array(B)

solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

solution

array([ 5.4,  1.4,  6.6,  2.6, -2.4])

In [12]:
# append 0 for fictional suppliers and receivers
alphas = np.append(solution[:2], 0)
betas = np.append(solution[2:], 0)

In [13]:
sales

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [14]:
def calculate_deltas(unit_profits, alphas, betas):
    # Step 5 in algorithm
    deltas = np.zeros(unit_profits.shape)
    for idx, row in enumerate(unit_profits[:-1][:-1]):
        for jdx, unit_profit in enumerate(row):
            deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 4)
    
    return deltas

In [25]:
deltas = calculate_deltas(unit_profits, alphas, betas)

In [16]:
sales

array([[10.,  0., 10.,  0.],
       [ 0., 28.,  2.,  0.],
       [ 0.,  0., 15., 50.]])

In [17]:
unit_profits * sales

array([[120.,   0.,  30.,   0.],
       [  0., 112.,  -2.,   0.],
       [  0.,   0.,   0.,   0.]])

In [18]:
def calculate_overall_return(unit_profits, sales):
    return np.sum(unit_profits[:-1, :-1] * sales[:-1, :-1])

def calculate_returns(unit_profits, sales):
    return unit_profits[:-1, :-1] * sales[:-1, :-1]

In [19]:
calculate_overall_return(unit_profits, sales)

260.0

In [20]:
calculate_returns(unit_profits, sales)

array([[120.,   0.,  30.],
       [  0., 112.,  -2.]])

In [21]:
class MiddleMan:
    def __init__(self, supp_idx=1, rec_idx=1):
        self.supp_idx = supp_idx
        self.rec_idx = rec_idx
        self.suppliers = []
        self.receivers = []
    
    def calculate(self):
        self.unit_profits = self.unit_profits_matrix()
        indices = self.calc_sorted_indices()
        
        supp_sum, dmnd_sum = self.supply_demand_sum()
        
        if supp_sum != dmnd_sum:
            self.unit_profits = np.pad(self.unit_profits, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.transport_cost_matrix = np.pad(self.transport_cost_matrix, ((0, 1), (0, 1)), 'constant', constant_values=0)
            self.suppliers.append(('DF', dmnd_sum, 0))
            self.receivers.append(('OF', supp_sum, 0))
        
        self.sales, supp, recv = self.sales_matrix()
        alphas, betas = self._calculate_alpha_beta()
        self.calculate_deltas(alphas, betas)
        
    def add_supplier(self, name, supply, cost):
        self.suppliers.append((name, supply, cost))
        
    def add_receiver(self, name, demand, price):
        self.receivers.append((name, demand, price))
        
    def add_transport_matrix(self, matrix:np.ndarray):
        self.transport_cost_matrix = matrix
        
    def unit_profits_matrix(self):
        un_pr_matrix = np.empty_like(self.transport_cost_matrix)
        # Suppliers
        for i, row in enumerate(self.transport_cost_matrix):
            # Receivers
            for j, num in enumerate(row):
                
                un_pr_matrix[i, j] = self.receivers[j][2] - self.suppliers[i][2] - self.transport_cost_matrix[i, j]
        
        return un_pr_matrix

    def supply_demand_sum(self):
        supp_sum = sum([supp[self.supp_idx] for supp in self.suppliers])
        dmnd_sum = sum([recv[self.rec_idx] for recv in self.receivers])
        return supp_sum, dmnd_sum
    
    def calc_sorted_indices(self):
        indices = np.argsort(self.unit_profits, axis=None)[::-1]
        indices = np.unravel_index(indices, self.unit_profits.shape)
        return indices
    
    def sales_matrix(self):
        suppliers_cpy = [list(tup) for tup in self.suppliers]
        receivers_cpy = [list(tup) for tup in self.receivers]
        # We start from REAL Suppliers and Receivers:
        sales = np.zeros(self.unit_profits.shape)
        for idx, jdx in zip(*indices):
            # Each iteration is one supplier-receiver transaction
            unit_profit = self.unit_profits[idx, jdx]
            supp_name, supply, cost = suppliers_cpy[idx]
            recv_name, demand, price = receivers_cpy[jdx]
            if supply == 0 or demand == 0:
                continue
            
            supply_after = supply - demand
            if supply_after >= 0:
                # no supply
                sales[idx, jdx] = demand
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = supply_after
                receivers_cpy[jdx][self.rec_idx]  = 0
            if supply_after < 0:
                # no demand
                sales[idx, jdx] = supply
                
                # Update
                suppliers_cpy[idx][self.supp_idx] = 0
                receivers_cpy[jdx][self.rec_idx]  = -supply_after
        
        # We go to FICTIONAL Suppliers and Receivers
        for idx, row in enumerate(sales):
            for jdx, num in enumerate(row):
                # Each iteration is one supplier-receiver transaction
                unit_profit = self.unit_profits[idx, jdx]
                supp_name, supply, cost = suppliers_cpy[idx]
                recv_name, demand, price = receivers_cpy[jdx]
                if supply == 0 or demand == 0:
                    continue
                
                supply_after = supply - demand
                if supply_after >= 0:
                    # no supply
                    sales[idx, jdx] = demand
                    
                    # Update
                    suppliers_cpy[idx][self.supp_idx] = supply_after
                    receivers_cpy[jdx][self.rec_idx]  = 0
                if supply_after < 0:
                    # no demand
                    sales[idx, jdx] = supply
                    
                    # Update
                    suppliers_cpy[idx][self.supp_idx] = 0
                    receivers_cpy[jdx][self.rec_idx]  = -supply_after
                
        if (sum([supp[self.supp_idx] for supp in suppliers_cpy]) or
            sum([recv[self.rec_idx] for recv in receivers_cpy])):
            raise Exception("Supply and Demand after calculations are not zero.")
        return sales, suppliers_cpy, receivers_cpy
    
    def _calculate_alpha_beta(self):
        mask = self.sales[:-1, :-1] !=0
        masked_array = (self.unit_profits[:-1, :-1] * mask)

        A = []
        B = []
        for i in range(mask.shape[0]):
            for j in range(mask.shape[1]):
                if mask[i, j]:
                    row = [0]*(mask.shape[0] + mask.shape[1])
                    row[i] = 1
                    row[mask.shape[0] + j] = 1
                    A.append(row)
                    B.append(masked_array[i, j])

        A = np.array(A)
        B = np.array(B)
        
        solution, residuals, rank, s = np.linalg.lstsq(A, B, rcond=None)

        alphas = np.append(solution[:2], 0)
        betas = np.append(solution[2:], 0)
        
        return alphas, betas
    
    def calculate_deltas(self, alphas, betas):
        # Step 5 in algorithm
        deltas = np.zeros(self.unit_profits.shape)
        for idx, row in enumerate(self.unit_profits[:-1][:-1]):
            for jdx, unit_profit in enumerate(row):
                deltas[idx, jdx] = round(unit_profit - alphas[idx] - betas[jdx], 4)
        
        return deltas
    
    def calculate_overall_return(self):
        return np.sum(self.unit_profits[:-1, :-1] * self.sales[:-1, :-1])

    def calculate_returns(self):
        return self.unit_profits[:-1, :-1] * self.sales[:-1, :-1]

In [22]:
mm = MiddleMan()
mm.add_supplier('D1', 20, 10)
mm.add_supplier('D2', 30, 12)

mm.add_receiver('O1', 10, 30)
mm.add_receiver('O2', 28, 25)
mm.add_receiver('O3', 27, 30)

transport_cost_matrix = np.array([
    [8, 14, 17],
    [12, 9, 19]
])

mm.add_transport_matrix(transport_cost_matrix)

mm.calculate()

In [23]:
mm.calculate_overall_return()

260.0

In [24]:
mm.calculate_returns()

array([[120.,   0.,  30.],
       [  0., 112.,  -2.]])